In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from utils import ours_read_csv
from sklearn.metrics import mean_squared_error


DIR_PATH_TOM = "C:\\Users\\ttomn\\OneDrive\\Desktop\\data-science-transportation-research"

In [24]:
file_name = "10000_streets_final_df.csv"

In [25]:
df = ours_read_csv(f"{DIR_PATH_TOM}\\{file_name}")
df

,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,...,Bridge,Tunnel,Boardwalk,PathTrail,StepStreet,Driveway,Ramp,Alley,U-Turn,FerryRoute
0,12,MULTILINESTRING ((-74.01213630770864 40.692040...,1,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
1,12,MULTILINESTRING ((-74.01213630770864 40.692040...,2,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,12,MULTILINESTRING ((-74.01213630770864 40.692040...,3,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,12,MULTILINESTRING ((-74.01213630770864 40.692040...,4,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,12,MULTILINESTRING ((-74.01213630770864 40.692040...,5,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286663,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,10,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286664,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,11,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286665,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286666,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,1,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [26]:
df = df.drop(columns=["the_geom", "ST_INDEX"])

In [27]:
# df[df[["With","Against", "Two-Way"]].sum(axis=1)==1]

In [28]:
df = df[(df["FerryRoute"]!=1)&(df["U-Turn"]!=1)]
df = df.drop(columns=["FerryRoute", "U-Turn"])
y = df.astype(np.float64)['COLLISIONS']
X = df.astype(np.float64).drop('COLLISIONS', axis=1)

In [29]:
for col in X.columns:
    if np.all((X[col]==1) | (X[col]==0)):
        continue
    X[col] = (X[col] - X[col].mean())/X[col].std()

In [30]:
X["SMALL_STREET"] = ((X["PathTrail"]==1)|(X["Boardwalk"]==1)|(X["StepStreet"]==1)|(X["Driveway"]==1)|\
                    (X["Alley"]==1)).astype(np.float64)
X.drop(columns=["PathTrail","Boardwalk","StepStreet","Driveway","Alley", "Street", ],
       inplace=True)
X

,MONTH,YEAR,Residential,Education,Cultural,Recreational,Social Services,Transportation,Commercial,Government,...,ST_WIDTH,ST_LENGTH,With,Against,Two-Way,Highway,Bridge,Tunnel,Ramp,SMALL_STREET
0,-1.559034,-1.579719,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,-0.971589,-0.554324,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,-1.271674,-1.579719,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,-0.971589,-0.554324,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-0.984314,-1.579719,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,-0.971589,-0.554324,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,-0.696955,-1.579719,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,-0.971589,-0.554324,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,-0.409595,-1.579719,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,-0.971589,-0.554324,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286663,1.027204,1.524128,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,0.304511,-1.045830,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1286664,1.314564,1.524128,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,0.304511,-1.045830,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1286665,1.601923,1.524128,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,0.304511,-1.045830,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1286666,-1.559034,1.834513,-0.100424,-0.127275,-0.054233,-0.130819,-0.101894,-0.083401,-0.079891,-0.045246,...,0.304511,-1.045830,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [36]:
freq_weights = None #y_train//2+1

In [37]:
model = sm.GLM(y_train, sm.tools.add_constant(X_train), family=sm.families.NegativeBinomial(), freq_weights=freq_weights)
result = model.fit(method="IRLS")

# Print the model summary
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             COLLISIONS   No. Observations:               958468
Model:                            GLM   Df Residuals:                   958435
Model Family:        NegativeBinomial   Df Model:                           32
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.1218e+05
Date:                Mon, 05 Jun 2023   Deviance:                   4.0057e+05
Time:                        21:09:37   Pearson chi2:                 1.84e+06
No. Iterations:                    30   Pseudo R-squ. (CS):            0.06444
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -2.66

In [38]:
y_pred = result.predict(sm.tools.add_constant(X_test))
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]

In [39]:
pd.DataFrame({'true':y_test, 'pred':y_pred})

,true,pred
336143,2.0,0.090097
334055,0.0,0.091492
964587,0.0,0.068169
1095495,0.0,0.005156
857636,0.0,0.116960
...,...,...
1206925,0.0,0.071705
1025438,0.0,0.004541
113410,0.0,0.097400
387433,0.0,0.092796


In [12]:
with open('output_10000.txt', 'w') as file:
    file.write(str(result.summary()) + '\n')
    file.write(f"normalized MSE on test data is {mse_test}")